# Ham or Spam?

🎯 The goal of this challenge is to classify emails as spams (1) or normal emails (0)

🧹 First, you will apply cleaning techniques to these textual data

👩🏻‍🔬 Then, you will convert the cleaned texts into a numerical representation

✉️ Eventually, you will apply the ***Multinomial Naive Bayes*** model to classify each email as either a spam or a regular email.

## (0) The NTLK library (Natural Language Toolkit)

In [ ]:
# !pip install nltk

In [1]:
# When importing nltk for the first time, we need to also download a few built-in libraries

import nltk

nltk.download('stopwords')
nltk.download('punkt')      # For nltk<3.9.0
nltk.download('punkt_tab')  # For nltk>=3.9.0
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/simonhingant/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/simonhingant/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/simonhingant/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/simonhingant/nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/simonhingant/nltk_data...


True

In [2]:
import pandas as pd

df = pd.read_csv("https://wagon-public-datasets.s3.amazonaws.com/05-Machine-Learning/10-Natural-Language-Processing/ham_spam_emails.csv")
df.head()

,text,spam
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1


## (1) Cleaning the (text) dataset

The dataset is made up of emails that are classified as ham [0] or spam[1]. You need to clean the dataset before training a prediction model.

### (1.1) Remove Punctuation

❓ Create a function to remove the punctuation. Apply it to the `text` column and add the output to a new column in the dataframe called `clean_text` ❓

In [3]:
import string

def remove_punctuation(text):
    for p in string.punctuation:
        text = text.replace(p, '')
    return text

# Apply function to text column
df["clean_text"] = df["text"].apply(remove_punctuation)

df[["text", "clean_text"]].head()


,text,clean_text
0,Subject: naturally irresistible your corporate...,Subject naturally irresistible your corporate ...
1,Subject: the stock trading gunslinger fanny i...,Subject the stock trading gunslinger fanny is...
2,Subject: unbelievable new homes made easy im ...,Subject unbelievable new homes made easy im w...
3,Subject: 4 color printing special request add...,Subject 4 color printing special request addi...
4,"Subject: do not have money , get software cds ...",Subject do not have money get software cds fr...


In [4]:
df

,text,spam,clean_text
0,Subject: naturally irresistible your corporate...,1,Subject naturally irresistible your corporate ...
1,Subject: the stock trading gunslinger fanny i...,1,Subject the stock trading gunslinger fanny is...
2,Subject: unbelievable new homes made easy im ...,1,Subject unbelievable new homes made easy im w...
3,Subject: 4 color printing special request add...,1,Subject 4 color printing special request addi...
4,"Subject: do not have money , get software cds ...",1,Subject do not have money get software cds fr...
...,...,...,...
5723,Subject: re : research and development charges...,0,Subject re research and development charges t...
5724,"Subject: re : receipts from visit jim , than...",0,Subject re receipts from visit jim thanks ...
5725,Subject: re : enron case study update wow ! a...,0,Subject re enron case study update wow all ...
5726,"Subject: re : interest david , please , call...",0,Subject re interest david please call shi...


### (1.2) Lower Case

❓ Create a function to lowercase the text. Apply it to `clean_text` ❓

In [5]:
def to_lowercase(text):
    return text.lower()

# Apply to the column 'clean_text'
df["clean_text"] = df["clean_text"].apply(to_lowercase)

df[["text", "clean_text"]].head()


,text,clean_text
0,Subject: naturally irresistible your corporate...,subject naturally irresistible your corporate ...
1,Subject: the stock trading gunslinger fanny i...,subject the stock trading gunslinger fanny is...
2,Subject: unbelievable new homes made easy im ...,subject unbelievable new homes made easy im w...
3,Subject: 4 color printing special request add...,subject 4 color printing special request addi...
4,"Subject: do not have money , get software cds ...",subject do not have money get software cds fr...


### (1.3) Remove Numbers

❓ Create a function to remove numbers from the text. Apply it to `clean_text` ❓

In [6]:
def remove_numbers(text):
    return ''.join(c for c in text if not c.isdigit())

# Apply to the column 'clean_text'
df["clean_text"] = df["clean_text"].apply(remove_numbers)

df[["text", "clean_text"]].head()


,text,clean_text
0,Subject: naturally irresistible your corporate...,subject naturally irresistible your corporate ...
1,Subject: the stock trading gunslinger fanny i...,subject the stock trading gunslinger fanny is...
2,Subject: unbelievable new homes made easy im ...,subject unbelievable new homes made easy im w...
3,Subject: 4 color printing special request add...,subject color printing special request addit...
4,"Subject: do not have money , get software cds ...",subject do not have money get software cds fr...


### (1.4) Remove StopWords

❓ Create a function to remove stopwords from the text. Apply it to `clean_text`. ❓

In [7]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    tokens = word_tokenize(text)
    tokens_clean = [word for word in tokens if word not in stop_words]
    return ' '.join(tokens_clean)

# Apply to 'clean_text'
df["clean_text"] = df["clean_text"].apply(remove_stopwords)

df[["text", "clean_text"]].head()


,text,clean_text
0,Subject: naturally irresistible your corporate...,subject naturally irresistible corporate ident...
1,Subject: the stock trading gunslinger fanny i...,subject stock trading gunslinger fanny merrill...
2,Subject: unbelievable new homes made easy im ...,subject unbelievable new homes made easy im wa...
3,Subject: 4 color printing special request add...,subject color printing special request additio...
4,"Subject: do not have money , get software cds ...",subject money get software cds software compat...


### (1.5) Lemmatize

❓ Create a function to lemmatize the text. Make sure the output is a single string, not a list of words. Apply it to `clean_text`. ❓

In [8]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

df["clean_text"] = df["clean_text"].apply(lambda x: ' '.join([lemmatizer.lemmatize(w) for w in x.split()]))

df[["text", "clean_text"]].head()



,text,clean_text
0,Subject: naturally irresistible your corporate...,subject naturally irresistible corporate ident...
1,Subject: the stock trading gunslinger fanny i...,subject stock trading gunslinger fanny merrill...
2,Subject: unbelievable new homes made easy im ...,subject unbelievable new home made easy im wan...
3,Subject: 4 color printing special request add...,subject color printing special request additio...
4,"Subject: do not have money , get software cds ...",subject money get software cd software compati...


In [9]:
df["clean_text"] = df["clean_text"].apply(
    lambda x: ' '.join([lemmatizer.lemmatize(w, pos='v') for w in x.split()])
)


In [10]:
df[["text", "clean_text"]].head()

,text,clean_text
0,Subject: naturally irresistible your corporate...,subject naturally irresistible corporate ident...
1,Subject: the stock trading gunslinger fanny i...,subject stock trade gunslinger fanny merrill m...
2,Subject: unbelievable new homes made easy im ...,subject unbelievable new home make easy im wan...
3,Subject: 4 color printing special request add...,subject color print special request additional...
4,"Subject: do not have money , get software cds ...",subject money get software cd software compati...


## (2) Bag-of-words Modelling

### (2.1) Digitizing the textual data into numbers

❓ Vectorize the `clean_text` to a Bag-of-Words representation with a default CountVectorizer. Save as `X_bow`. ❓

In [11]:
from sklearn.feature_extraction.text import CountVectorizer

# Vectorization (Bag-of-Words)
vectorizer = CountVectorizer()
X_bow = vectorizer.fit_transform(df["clean_text"])



In [14]:

# visualiser les 10 premières features
print(vectorizer.get_feature_names_out()[:10])


['aa' 'aaa' 'aaaenerfax' 'aadedeji' 'aagrawal' 'aal' 'aaldous' 'aaliyah'
 'aall' 'aanalysis']


In [13]:
print(X_bow.shape)

(5728, 28112)


### (2.2) Multinomial Naive Bayes Modelling

❓ Cross-validate a MultinomialNB model with the bag-of-words data. Score the model's accuracy. ❓

In [15]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score

# Définition du modèle
model = MultinomialNB()

# Validation croisée sur le Bag-of-Words
scores = cross_val_score(model, X_bow, df["spam"], cv=5, scoring="accuracy")

# Résultat
print("Accuracy moyenne :", round(scores.mean(), 3))


Accuracy moyenne : 0.988


In [16]:
import numpy as np
import pandas as pd

# Refit le modèle sur toutes les données
model.fit(X_bow, df["spam"])

# Récupère les mots du vocabulaire
words = np.array(vectorizer.get_feature_names_out())

# Importance = probabilité conditionnelle du mot dans le spam
spam_importance = model.feature_log_prob_[1]
ham_importance = model.feature_log_prob_[0]

# Classement des mots
top_spam = words[np.argsort(spam_importance)[-20:]]  # 20 plus liés aux spams
top_ham = words[np.argsort(ham_importance)[-20:]]    # 20 plus liés aux hams

print("Top mots SPAM :", top_spam)
print("Top mots HAM  :", top_ham)


Top mots SPAM : ['order' 'send' 'click' 'one' 'time' 'market' 'http' 'free' 'please'
 'money' 'receive' 'information' 'get' 'make' 'email' 'business' 'mail'
 'com' 'company' 'subject']
Top mots HAM  : ['price' 'work' 'know' 'group' 'time' 'meet' 'research' 'forward' 'pm'
 'thank' 'cc' 'would' 'com' 'please' 'kaminski' 'hou' 'vince' 'subject'
 'ect' 'enron']


🏁 Congratulations !

💾 Don't forget to git add/commit/push your notebook...

🚀 ... and move on to the next challenge !